In [2]:
import os
os.chdir('../')


In [3]:
%pwd

'd:\\Cdac_ML\\Assignments\\Lab_assesment'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [7]:
from src.ML_LAB.constants import * # Apne import path check kar lena
from src.ML_LAB.utils.common import read_yaml, create_directories, save_json
from src.ML_LAB.entity import *
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Config_yaml_path,
        params_filepath = params_yaml_path,
        schema_filepath = schema_yaml_path):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier # Modified for RF
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
        )

        return model_evaluation_config

In [8]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib
from src.ML_LAB.utils.common import save_json
from src.ML_LAB.entity import *
from pathlib import Path

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        # Classification Metrics
        acc = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, average='weighted') # weighted handles imbalance if any
        recall = recall_score(actual, pred, average='weighted')
        f1 = f1_score(actual, pred, average='weighted')
        return acc, precision, recall, f1

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]
        
        predicted_qualities = model.predict(test_x)

        (acc, precision, recall, f1) = self.eval_metrics(test_y, predicted_qualities)
        
        # Saving metrics as local JSON
        scores = {
            "accuracy": acc, 
            "precision": precision, 
            "recall": recall, 
            "f1_score": f1
        }
        
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2025-11-29 11:09:53,262: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-29 11:09:53,270: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-29 11:09:53,284: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-29 11:09:53,284: INFO: common: created directory at: artifacts]
[2025-11-29 11:09:53,284: INFO: common: created directory at: artifacts/model_evaluation]
[2025-11-29 11:09:55,227: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
